In [22]:
from google.colab import files
files.upload()

Saving websites.csv to websites.csv


{'websites.csv': b'1,"fonts.googleapis.com",10\n2,"facebook.com",10\n3,"twitter.com",10\n4,"google.com",10\n5,"youtube.com",10\n6,"s.w.org",10\n7,"instagram.com",10\n8,"googletagmanager.com",10\n9,"linkedin.com",10\n10,"ajax.googleapis.com",10\n11,"plus.google.com",10\n12,"gmpg.org",10\n13,"pinterest.com",9.63\n14,"fonts.gstatic.com",9.6\n15,"wordpress.org",9.54\n16,"en.wikipedia.org",9.54\n17,"youtu.be",9.47\n18,"maps.google.com",9.3\n19,"itunes.apple.com",9.21\n20,"github.com",9.18\n21,"bit.ly",9.11\n22,"play.google.com",9.07\n23,"goo.gl",9.03\n24,"docs.google.com",9.02\n25,"cdnjs.cloudflare.com",8.99\n26,"vimeo.com",8.98\n27,"support.google.com",8.87\n28,"google-analytics.com",8.8\n29,"maps.googleapis.com",8.79\n30,"flickr.com",8.76\n31,"vk.com",8.74\n32,"t.co",8.72\n33,"reddit.com",8.69\n34,"amazon.com",8.66\n35,"medium.com",8.64\n36,"sites.google.com",8.57\n37,"drive.google.com",8.51\n38,"creativecommons.org",8.47\n39,"microsoft.com",8.47\n40,"developers.google.com",8.46\n41,"adob

In [29]:
import pandas as pd
data = pd.read_csv("websites.csv")
data.head()

,idx,URLs,rate
0,2,facebook.com,10.0
1,3,twitter.com,10.0
2,4,google.com,10.0
3,5,youtube.com,10.0
4,6,s.w.org,10.0


In [40]:
#collecting 1000 legitimate urls randomly for testing
data.columns = ['idx', 'URLs', 'rate']
valid_urls = data.sample(n = 500, random_state = 300).copy()
valid_urls = valid_urls.reset_index(drop=True)
valid_urls.head(4)

,idx,URLs,rate
0,541,flaticon.com,6.77
1,618,notion.so,6.70
2,319,samsung.com,7.06
3,160,google.ca,7.48


In [4]:
!pip uninstall whois
!pip install python-whois

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-whois: filename=python_whois-0.8.0-py3-none-any.whl size=103247 sha256=fd5dde3f8de7428b931e093ccd268ed3470744564627b8776c14779ac0ddce4f
  Stored in directory: /root/.cache/pip/wheels/10/f1/87/145023b9a206e2e948be6480c61ef3fd3dbb81ef11b6977782
Successfully built python-whois


In [41]:
total = 2000

In [42]:
import re
import whois
from datetime import datetime
import requests
import argparse
import ssl
import certifi
import socket



def get_domain(url):
  domain = url.split("//")[-1].split("/")[0]
  if re.match(r"^www.",domain):
    domain = domain.replace("www.","")
  #print(domain)
  return domain

# url feature:
def url_length(url):
  if len(url) < 54:
      length = 0
  else:
      length = 1
  return length


def contains_ip_address(url):
  return int(bool(re.search(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', url)))

def _ssl_certificate(url):
  domain = get_domain(url)

  flag = 0
  try:
    response = requests.get(url, timeout=1)
    if response.status_code == 200:
      if response.url.startswith("https://"):
        flag = 1
  except:
    return 0

  if flag == 0:
    return 0

  context = ssl.create_default_context(cafile=certifi.where())

  try:
    with socket.create_connection((domain, 443)) as sock:
      with context.wrap_socket(sock, server_hostname=domain) as ssock:
        cert = ssock.getpeercert()

        self_signed = 0
        expired = 0
        # Check if the certificate is self-signed
        if cert['issuer'] == cert['subject']:
          self_signed = 1

        # Check if the certificate has expired
        not_after_str = cert['notAfter']
        not_after = ssl.cert_time_to_seconds(not_after_str)
        if not_after < ssl.time.time():
          expired = 1

        if expired and self_signed:
          return 1 #phishing
        else:
          return 0

  except ssl.CertificateError:
    # failed to verify its own public key
    return 1
  #except ssl.SSLError as e:
      # suspicious, but skip for now
  except:
    return 1


def keyword_match(url):
    return int("login" in url)  # 1 if keyword exists, 0 if not

def special_char(url):
    phishing = 0
    if "_" in url:
        phishing = 1
    if "," in url:
        phishing = 1
    if ";" in url:
        phishing = 1
    return phishing



#domain featur

def domain_exp_date(url):
  try:
    domain = get_domain(url)
    info = whois.whois(domain)
    expiration_date = info.expiration_date

    if isinstance(expiration_date, list):
      expiration_date = min(expiration_date)

    if expiration_date:
      days_until_expiration = (expiration_date - datetime.now()).days
      if days_until_expiration <= 180:
        return 1
      else:
        return 0
    else:
      return 0
  except:
    return 0

def domain_creation_date(url):
  try:
    domain = get_domain(url)
    info = whois.whois(domain)
    creation_date = info.creation_date

    if isinstance(creation_date, list):
      creation_date = min(creation_date)

    if creation_date:
      days_created = (datetime.now()-creation_date).days
      if days_created <= 365:
        return 1
      else:
        return 0
    else:
      return 0
  except:
    return 0

def domain_age(url):
    try:
        domain = get_domain(url)

        # Perform a WHOIS lookup to get domain information
        domain_info = whois.whois(domain)

        # Extract the creation/expiration date from the WHOIS data
        creation_date = domain_info.creation_date
        expiration_date = domain_info.expiration_date

        if isinstance(creation_date, list):
            creation_date = creation_date[0]

        # Calculate domain age
        if creation_date:
            age = (expiration_date - creation_date).days
            # if age of domain < 6 months
            if_phishing = 1
            if ((age/30) < 6):
              if_phishing = 1
            else:
              if_phishing = 0
            return if_phishing
        else:
            return 0
    except:
        return 0

def contains_at(url):
    domain = get_domain(url)
    return int("@" in domain) # return 1 if @ contains (phishing), 0 otherwise

def dots_count(url):
    domain = get_domain(url)
    #The number of dots in the domain part of the URL represents the number of subdomains.
    dot_count = domain.count('.')
    if dot_count <= 3:
        return 0
    else:
        return 1

def slashes_count(url):
    domain = get_domain(url)
    slash_count = domain.count('/')
    if slash_count <= 3:
        return 0
    else:
        return 1


#html features:
def count_redirect_urls(url):
# The more redirect URLs are in a webpage the more malicious the URL is.
    try:
        response = requests.get(url, allow_redirects=True, timeout=2)

        redirect_count = len(response.history)

        if redirect_count <= 1:
            return 0
        else:
            return 1

    except:
        return 0


def right_click_disable(url):
    try:
        response = requests.get(url, allow_redirects=True, timeout=2)
        if re.findall(r"event.button ?== ?2", response.text):
            return 0
        else:
            return 1
    except:
        return 0


def status_bar_change(url):
    try:
        response = requests.get(url, timeout=2)
        if 'window.status' in response.text:
            return 1
        else:
            return 0
    except:
        return 0


# Sample URL for analysis
# url = "https://drive.google.com/file/d/1gGAMknkZAiK8m01vtpQJebwuTdoxAMVP/view?usp=sharing"



# Define heuristics and their weights
# ref: https://drive.google.com/file/d/1gGAMknkZAiK8m01vtpQJebwuTdoxAMVP/view?usp=sharing, pp.26
heuristics = {
    #'URL Length': 0,
    #'Contains IP Address': 0.09,
    'SSL Certificate': 3,
    #'Keyword Match': 0.1,
    'Special Char': 1,
    'Domain Age': 4,
    'Contains At': 1,
    'Dots Count': 1,
   # 'Slashes Count': 0,
    #'Redirect URLs': 0,
    'Right Click Disable': 1,
    #'Status Bar Change': 0,
    'Exp Date': 2,
    'Creation_Date': 5
}

In [43]:
counter = 0
for i in range(100, 200):

  url = valid_urls['URLs'][i]

  criteria = {
    #  'URL Length': url_length(url),
      #'Contains IP Address': contains_ip_address(url),
      'SSL Certificate': _ssl_certificate(url),
      #'Keyword Match': keyword_match(url),
      'Special Char': special_char(url),
      'Domain Age': domain_age(url),
      'Contains At': contains_at(url),
      'Dots Count': dots_count(url),
   #   'Slashes Count': slashes_count(url),
      #'Redirect URLs': count_redirect_urls(url),
      'Right Click Disable': right_click_disable(url),
    #  'Status Bar Change': status_bar_change(url)
      'Exp Date': domain_exp_date(url),
      'Creation_Date': domain_creation_date(url)
  }

  # Calculate the phishing score
  phishing_score = sum(heuristics[heuristic] * criteria[heuristic] for heuristic in heuristics)

  # a threshold (adjust as needed)
  threshold = 2.9

  # Determine if the URL is potentially phishing
  is_phishing = phishing_score > threshold

  # Output the result
  if is_phishing == 0:
      counter += 1



Error trying to connect to socket: closing socket - timed out
Error trying to connect to socket: closing socket - timed out
Error trying to connect to socket: closing socket - timed out
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - [Errno -2] Name or service not known
Error trying to connect to socket: closing socket - timed out


In [39]:
counter

100